In [ ]:
# set your project directory in google drive
BASE_DIR = "/content/drive/MyDrive/"
%cd {BASE_DIR}

### Installing Packages

- ultralytics

In [2]:
!pip install ultralytics==8.0.157

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.5 MB/s eta 0:00:00


In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.157 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


- supervision

In [4]:
!pip install supervision==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 873.8 kB/s eta 0:00:00


### Imports

In [27]:
from ultralytics import YOLO
from tqdm.notebook import tqdm
from supervision.geometry.core import Point
from supervision.utils.video import VideoSink
from supervision.utils.video import VideoInfo
from supervision.utils.notebook import plot_image
from supervision.detection.core import Detections
from supervision.detection.annotate import BoxAnnotator
from supervision.draw.color import Color, ColorPalette
from supervision.tracker.byte_tracker.core import ByteTrack
from supervision.utils.video import get_video_frames_generator
from supervision.detection.line_counter import LineZone, LineZoneAnnotator

### Download video for experimentation

In [ ]:
%cd {BASE_DIR}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

In [19]:
VIDEO_PATH_1 = f"{BASE_DIR}/vehicle-counting.mp4"

## Load YoloV8 model

In [20]:
model = YOLO("yolov8x.pt")

model.fuse()

label_map = model.model.names

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


In [ ]:
!yolo task=detect mode=predict source={VIDEO_PATH_1}

## Initialize ByteTracker

In [22]:
byte_tracker = ByteTrack()

### Perform Detection - Tracking - Counting

In [23]:
RESULT_PATH = f"{BASE_DIR}/vehicle_counting_result.mp4"

In [24]:
LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)

line_counter = LineZone(start=LINE_START,end=LINE_END)
line_annotator = LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

- detection - tracking - counting

In [44]:
generator = get_video_frames_generator(VIDEO_PATH_1)

box_annotator = BoxAnnotator(color=ColorPalette.default(),thickness=4, text_thickness=4,text_scale=2)

video_info = VideoInfo.from_video_path(VIDEO_PATH_1)
with VideoSink(RESULT_PATH, video_info) as sink:
  for frame in tqdm(generator,total=video_info.total_frames):

    results = model(frame)[0]
    detections = Detections.from_ultralytics(results)

    detections = byte_tracker.update_with_detections(detections=detections)
    labels=[f"{label_map[class_id]} {confidence:0.2f} -track_id:{tracker_id}" for _,_,confidence,class_id,tracker_id in detections]

    line_counter.trigger(detections=detections)

    annotated_frame = box_annotator.annotate(scene=frame, detections=detections,labels=labels)
    line_annotator.annotate(frame=frame,line_counter=line_counter)

    sink.write_frame(frame)
    #show_frame_in_notebook(annotated_frame,(10,10))

  0%|          | 0/538 [00:00<?, ?it/s]


0: 384x640 3 cars, 1 truck, 63.3ms
Speed: 4.4ms preprocess, 63.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 64.4ms
Speed: 4.8ms preprocess, 64.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 trucks, 63.2ms
Speed: 11.1ms preprocess, 63.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 69.9ms
Speed: 3.4ms preprocess, 69.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 63.5ms
Speed: 4.0ms preprocess, 63.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 63.4ms
Speed: 3.2ms preprocess, 63.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 64.2ms
Speed: 6.4ms preprocess, 64.2ms inference, 10.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 69.0ms
Speed: 3.3ms preprocess, 69.0m